In [1]:
using POMDPs
using Random # for AbstractRNG
using POMDPModelTools
using Pkg

using DataFrames
Pkg.add("JSON")

POMDPs.add_registry()
using Pkg; Pkg.add("DiscreteValueIteration")

using DiscreteValueIteration

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`


┌ Warning: Some registries failed to update:
│     — /Users/efan/.julia/registries/General — failed to fetch from repo
│     — /Users/efan/.julia/registries/JuliaPOMDP — failed to fetch from repo
└ @ Pkg.API /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Pkg/src/API.jl:144


 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
   Cloning registry from "https://github.com/JuliaPOMDP/Registry"
Fetching: [========================================>]  100.0 %.0 %=========>                               ]  21.4 %]  42.7 %==========================>              ]  64.1 %]  85.3 %> ]  96.6 %

┌ Info: registry `JuliaPOMDP` already exists in `~/.julia/registries`
└ @ POMDPs /Users/efan/.julia/packages/POMDPs/JiYXY/src/utils.jl:65


 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [2]:
using Pkg
Pkg.add("StaticArrays")
using POMDPSimulators
using POMDPPolicies

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [3]:

include("./ParallelChains.jl")

setup_agents (generic function with 1 method)

In [ ]:
num_chains = 5
num_states = 6
r_mean = 5.0
r_std = 2.0

true_values = DataFrame(theta = [r_mean for i in 1:num_chains],
                        std = [r_std  for i in 1:num_chains])

In [ ]:
using POMDPSimulators
using POMDPPolicies
rng = MersenneTwister(1235);

PParallelChainMDP() = PParallelChainMDP(num_states+1,num_chains, .9,

        true_values.theta,
        true_values.std,
        MersenneTwister(1235))
m = PParallelChainMDP()

# policy that maps every input to an action
policy = FunctionPolicy(s->3)
for i in 1:3
    for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
      render(m, (s,a,r))
      println("s,a,r:($s,$a,$r)")
    end
end

In [ ]:
curry(f, a) = (xs...) -> f(a, xs...)

In [ ]:
using POMDPSimulators
using POMDPPolicies
rng = MersenneTwister(1235);

num_chains = 10
num_states = 30
r_mean = 5.0
r_std = 2.0
@requirements_info ValueIterationSolver() PParallelChainMDP(num_states+1,num_chains, .9,

        randn(rng, Float32, num_chains) .* r_std .+ r_mean)

In [ ]:
num_chains=2
r_mean = 0
r_std = 1
## sigma is 1
## Reward is noisy ~ N(theta_c, sigma^2)
## Initial prior mean is N(theta_c, sigma^2 + c)
rng = MersenneTwister(1235)
th_c = [randn(rng, Float64, 1)[1] * (100 + i) for i in 1:num_chains]
println(th_c)
true_values = DataFrame(theta = th_c,
                         std = [r_std  for i in 1:num_chains])
mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        true_values.theta,
        true_values.std, 
        rng)
# @requirements_info ValueIterationSolver() mdp
solver = ValueIterationSolver(max_iterations=100, belres=1e-6, include_Q=true, verbose=true) # initializes the Solver type
vip = solve(solver, mdp) # runs value iterations


In [ ]:
vip.qmat

In [ ]:
LinearIndices((num_chains, num_states))[1,1]

In [ ]:

for i in 1:num_chains
    println("q value for (1,1),$i is $(vip.qmat[LinearIndices((num_chains, num_states))[1,1],i])")
    println("q value for $((i,num_states)),1 is $(vip.qmat[LinearIndices((num_chains, num_states))[i,num_states],1])")
end

In [ ]:
mdp.Rs

In [ ]:
using POMDPSimulators
using POMDPPolicies



for (s, a, r) in stepthrough(mdp, vip, "s,a,r", max_steps=40)
    render(mdp, (s,a,r))
    println("s,a,r:($s,$a,$r)")
end



In [4]:
function ucb_pol(li, policy, priors, i, actions, s)
    best_action = action(policy, s)
    #println("in state: $s, best action:$best_action")
    return action(policy, s)
end

ucb_pol (generic function with 1 method)

In [ ]:
# li = LinearIndices((num_chains, num_states))
function ucb_update_priors(priors, history, i)
    return priors
end

In [5]:
curry(f, x) = (xs...) -> f(x, xs...)
function ucb_mdp_builder(true_mdp, priors, i, num_states, num_chains, steps)
  # build mdp
  new_means = priors.theta + priors.std
  rng = MersenneTwister(1233)
  #println("new means: $new_means, std: $(priors.std), num_chains:$num_chains, num_states:$num_states, i:$i, mdp:$(typeof(true_mdp))")
  # (randn(p.rng, Float32, 1) .* p.Rs_means[s[1]]  .+ p.Rs_means[s[1]])[1]
  mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        [randn(rng, Float32,1)[1] * priors.theta[j] + priors.std[j] for j in 1:num_chains])
  #println("#2")
  # solve mdp
  #println("#3")
  solver = ValueIterationSolver(max_iterations=1000, belres=1e-6, include_Q=true)#, verbose=true) # initializes the Solver type
  vip = solve(solver, mdp)
  li = LinearIndices((num_chains, num_states))
  #print("policy q: $(vip.qmat)")
  new_policy = FunctionPolicy(curry(curry(curry(curry(curry(ucb_pol, li), vip), priors),i), actions))
  # print("new policy: $new_policy")
  # return iterator solved mdp
  return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, "s,a,r,sp,t",
                            max_steps=steps))
end

ucb_mdp_builder (generic function with 1 method)

In [ ]:

do_update_priors() =  false
update_priors() = false

In [33]:

include("./ParallelChains.jl")
using Distributions
function do_runs(nruns, num_agents, num_chains, num_states, H, update_priors_check, update_priors, mdp_iter_builder)
    # UCB

    # setup constants here
    num_agents = num_agents
    num_chains = num_chains
    num_states = num_states
    epochs = 1
    nruns = nruns
    H = H

    # So best idea:
    # for ucb:
    # we generate an mdp with mu + sigma rewards (priors) for each end node.
    #  we collect history until the criteria in PAC-EXPLORE then update the prior
    # 
    # for thompson sampling
    # at each time step (might skip this for this problem and save for max rew path) 
    #   take rewards and use to create a posterior.
    # for seed sampling
    # at beginning of episode
    #   each agent generates a new random "seed" 
    #   at each time step
    #     each agent generates a new mdp based on a deterministic mapping from seed to rewards (which also takes
    #              history into account)
    # this needs to update the actual reward.

    ## PC notes
    ## theta_c ~ N(0, 100 + c)
    r_mean = 0
    r_std = 1
    ## sigma is 1
    ## Reward is noisy ~ N(theta_c, sigma^2)
    ## Initial prior mean is N(theta_c, sigma^2 + c)
    rng = MersenneTwister(1236)
    base = 10 # 100  # lowered a bit for testing
    runs = []
    true_values_list = DataFrame( run = [],
                                 c = [],
                                 theta = [],
                                 std = [])
    for run in 1:nruns
        th_c = [randn(rng, Float64, 1)[1] * sqrt(base + i) for i in 1:num_chains]
        #th_c[1] = 300
        println(th_c)
        true_values = DataFrame( run = [run for i in 1:num_chains],
                                 c = [i for i in 1:num_chains],
                                 theta = th_c,
                                 std = [r_std  for i in 1:num_chains])
        print("true mdp: $true_values")
        priors = DataFrame(theta = [true_values.theta[i] for i in 1:num_chains],
                           std = [r_std + i for i in 1:num_chains],
                           state = [(chain,num_states) for chain in 1:num_chains])
        print("priors: $priors")
        mdp = PParallelChainMDP(num_states+1,num_chains, .9,
                true_values.theta)
        do_update_priors() =  false
        # priors = randn(rng, Float32, num_chains) .* (r_std + c) .+ r_mean
        hist = run_chain!(
                   mdp_iter_builder=mdp_iter_builder,
                   true_mdp=mdp,
                   do_update_priors=update_priors_check,
                   update_priors=update_priors,
                   priors=priors,
                   n_agents=num_agents,
                   num_states=num_states,
                   num_chains=num_chains,
                   epochs=epochs,
                   steps=H) #,
                   #rev_action_map=rev_action_map)
        #print(Q_tables)
        println("(e,i,t,st,r)")
        hist
        for (e, ag, t, st, r)  in hist
          push!(runs, (run, e, ag, t, st, r)) 
        end
        append!(true_values_list, true_values)

    end
    return (true_values_list, runs)
end

do_runs (generic function with 1 method)

In [34]:
num_agents = 100
num_chains = 10
num_states = 10
epochs = 1
nruns = 1
H = 30
do_update_priors() = true
true_values_list, runs = do_runs(nruns, num_agents, num_chains, num_states, H, do_update_priors, ucb_update_priors, ucb_mdp_builder)
true_values_list, runs

[-4.54912, 0.815592, -0.576327, 2.55318, -3.81941, 2.93883, -2.47224, 0.579177, -7.63242, 1.48565]
true mdp: 10×4 DataFrame
│ Row │ run   │ c     │ theta     │ std   │
│     │ Int64 │ Int64 │ Float64   │ Int64 │
├─────┼───────┼───────┼───────────┼───────┤
│ 1   │ 1     │ 1     │ -4.54912  │ 1     │
│ 2   │ 1     │ 2     │ 0.815592  │ 1     │
│ 3   │ 1     │ 3     │ -0.576327 │ 1     │
│ 4   │ 1     │ 4     │ 2.55318   │ 1     │
│ 5   │ 1     │ 5     │ -3.81941  │ 1     │
│ 6   │ 1     │ 6     │ 2.93883   │ 1     │
│ 7   │ 1     │ 7     │ -2.47224  │ 1     │
│ 8   │ 1     │ 8     │ 0.579177  │ 1     │
│ 9   │ 1     │ 9     │ -7.63242  │ 1     │
│ 10  │ 1     │ 10    │ 1.48565   │ 1     │priors: 10×3 DataFrame
│ Row │ theta     │ std   │ state    │
│     │ Float64   │ Int64 │ Tuple…   │
├─────┼───────────┼───────┼──────────┤
│ 1   │ -4.54912  │ 2     │ (1, 10)  │
│ 2   │ 0.815592  │ 3     │ (2, 10)  │
│ 3   │ -0.576327 │ 4     │ (3, 10)  │
│ 4   │ 2.55318   │ 5     │ (4, 10)  │
│ 5   │ -

e: 1, t: 10, agent 79, result: (s = [9, 10], a = 1, r = -7.632423786961385, sp = [9, 11], t = 10)
agent 79 is done
Updating priors
e: 1, t: 10, agent 80, result: (s = [9, 10], a = 1, r = -7.632423786961385, sp = [9, 11], t = 10)
agent 80 is done
Updating priors
e: 1, t: 10, agent 81, result: (s = [9, 10], a = 1, r = -7.632423786961385, sp = [9, 11], t = 10)
agent 81 is done
Updating priors
e: 1, t: 10, agent 82, result: (s = [9, 10], a = 1, r = -7.632423786961385, sp = [9, 11], t = 10)
agent 82 is done
Updating priors
e: 1, t: 10, agent 83, result: (s = [9, 10], a = 1, r = -7.632423786961385, sp = [9, 11], t = 10)
agent 83 is done
Updating priors
e: 1, t: 10, agent 84, result: (s = [9, 10], a = 1, r = -7.632423786961385, sp = [9, 11], t = 10)
agent 84 is done
Updating priors
e: 1, t: 10, agent 85, result: (s = [9, 10], a = 1, r = -7.632423786961385, sp = [9, 11], t = 10)
agent 85 is done
Updating priors
e: 1, t: 10, agent 86, result: (s = [9, 10], a = 1, r = -7.632423786961385, sp = [9

(10×4 DataFrame
│ Row │ run │ c   │ theta     │ std │
│     │ Any │ Any │ Any       │ Any │
├─────┼─────┼─────┼───────────┼─────┤
│ 1   │ 1   │ 1   │ -4.54912  │ 1   │
│ 2   │ 1   │ 2   │ 0.815592  │ 1   │
│ 3   │ 1   │ 3   │ -0.576327 │ 1   │
│ 4   │ 1   │ 4   │ 2.55318   │ 1   │
│ 5   │ 1   │ 5   │ -3.81941  │ 1   │
│ 6   │ 1   │ 6   │ 2.93883   │ 1   │
│ 7   │ 1   │ 7   │ -2.47224  │ 1   │
│ 8   │ 1   │ 8   │ 0.579177  │ 1   │
│ 9   │ 1   │ 9   │ -7.63242  │ 1   │
│ 10  │ 1   │ 10  │ 1.48565   │ 1   │, Any[(1, 1, 1, 1, [1, 1], 0), (1, 1, 1, 2, [10, 2], 0), (1, 1, 2, 1, [1, 1], 0), (1, 1, 1, 3, [10, 3], 0), (1, 1, 2, 2, [10, 2], 0), (1, 1, 3, 1, [1, 1], 0), (1, 1, 1, 4, [10, 4], 0), (1, 1, 2, 3, [10, 3], 0), (1, 1, 3, 2, [10, 2], 0), (1, 1, 4, 1, [1, 1], 0)  …  (1, 1, 97, 10, [9, 10], -8.46579), (1, 1, 98, 9, [9, 9], 0), (1, 1, 99, 8, [9, 8], 0), (1, 1, 100, 7, [9, 7], 0), (1, 1, 98, 10, [9, 10], -8.07567), (1, 1, 99, 9, [9, 9], 0), (1, 1, 100, 8, [9, 8], 0), (1, 1, 99, 10, [9, 10], 

In [6]:

using DataFrames
using Statistics
function ucb_update_priors(priors, hist)
    # each state must be visted me times to update the prior
    # me is # of S*D^2 s= states, D =diameter (https://www.aaai.org/ocs/index.php/AAAI/AAAI15/paper/view/9542/9919)
    #     (D defined in http://www.jmlr.org/papers/volume11/jaksch10a/jaksch10a.pdf)
    ## ^^^ Actually, think should just do this statistically, and if n > 2, then we can adjust the prior.
    df = DataFrame(epoch = [x[1] for x in hist], agent=[x[2] for x in hist],
               time = [x[3] for x in hist], chain = [x[4][1] for x in hist], chain_state = [x[4][2] for x in hist],
        reward = [x[5] for x in hist])
    new_theta = []
    new_std = []
    new_state = []
    for arow in eachrow(priors)
        if nrow(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:]) > 1
            # print("df subset:")
            # println(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:])
            # new mean = (s_p^2 /(s_s^2 + s_p^2 )) * mu_s + (s_s^2 /(s_s^2 + s_p^2 )) * mu_p
            # new std = 1/(1/(s_p^2) + 1/(s_s^2)
            mu_p = arow.theta
            s_p_2 = arow.std^2
            mu_s = mean(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:].reward)
            s_s_2 = var(df[(df.chain.==arow.state[1]).&(df.chain_state.==arow.state[2]),:].reward)
            # println("mu_p:$mu_p,s_p_2:$s_p_2,mu_s:$mu_s,s_s_2:$s_s_2")
            tmp_theta =  (s_p_2/(s_s_2 + s_p_2)) * mu_s + (s_s_2/(s_s_2 + s_p_2)) * mu_p
            tmp_std = 1/(1/s_p_2 + 1/s_s_2)
        else
            tmp_theta =  arow.theta
            tmp_std = arow.std
        end
        push!(new_state, arow.state)
        push!(new_theta, tmp_theta)
        push!(new_std, tmp_std)
    end
    new_priors = DataFrame(theta=new_theta, std=new_std, state=new_state)
    #println("new priors: $new_priors")
    return new_priors
end



ucb_update_priors (generic function with 1 method)

In [ ]:
println(priors)
ucb_update_priors(priors, hist)

In [54]:
function get_average_regret(results, chain_len, num_chains, theta)
    df = DataFrame(run = [x[1] for x in results], epoch = [x[2] for x in results], agent=[x[3] for x in results],
               time = [x[4] for x in results], c = [x[5][1] for x in results], state = [x[5][1] for x in results],
               reward = [x[6] for x in results])
    df_R = df[df.reward.>0,:]
    th_max = by(theta, :run, :theta => maximum)
    theta_joined = join(theta, th_max, on = :run)
    df_joined = join(df_R, theta_joined, on = [:run,:c] )
    # needs to be max theta
    df_joined.Regret = df_joined.theta - df_joined.reward
    df_joined
end

get_average_regret (generic function with 2 methods)

In [55]:
true_values_list

,run,c,theta,std
,Any,Any,Any,Any
1,1,1,-4.54912,1
2,1,2,0.815592,1
3,1,3,-0.576327,1
4,1,4,2.55318,1
5,1,5,-3.81941,1
6,1,6,2.93883,1
7,1,7,-2.47224,1
8,1,8,0.579177,1
9,1,9,-7.63242,1


In [56]:

chain_len = num_states
num_chains = num_chains
get_average_regret(runs, chain_len, num_chains, true_values_list)

,run,epoch,agent,time,c,state,reward,theta,std,theta_maximum,Regret
,Int64,Int64,Int64,Int64,Int64,Int64,Real,Any,Any,Float64,Float64
1,1,1,1,10,10,10,2.353,1.48565,1,2.93883,-0.867347
2,1,1,2,10,10,10,0.583911,1.48565,1,2.93883,0.901744
3,1,1,3,10,10,10,0.991176,1.48565,1,2.93883,0.494479
4,1,1,4,10,10,10,0.58274,1.48565,1,2.93883,0.902914
5,1,1,5,10,10,10,2.35006,1.48565,1,2.93883,-0.864401
6,1,1,6,10,10,10,3.69753,1.48565,1,2.93883,-2.21188
7,1,1,7,10,10,10,2.01847,1.48565,1,2.93883,-0.532813
8,1,1,8,10,10,10,1.21392,1.48565,1,2.93883,0.271735
9,1,1,9,10,10,10,1.98799,1.48565,1,2.93883,-0.502334
